In [11]:
#Import Dependencies
from pylab import *
import os
import pandas as pd
# import librosa
# import librosa.display
# import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

# Model

In [2]:
df = pd.read_csv('../data/voice_data.csv')
df.head()

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,label,emotions
0,-573.379150,34.064041,-33.337578,3.489916,-13.482152,-6.327230,-17.734171,-15.280939,-8.379515,-0.323866,...,-1.091130,-2.829661,-1.520689,-2.216941,-3.051777,1.104535,0.969608,2.822009,6,fearful
1,-545.244568,38.073475,-8.023163,7.236882,-13.688817,-2.157916,-12.315373,-7.948586,-14.858138,1.719064,...,-0.281293,0.684120,0.073632,2.641415,0.838373,1.800676,-0.601248,1.839665,5,angry
2,-690.535461,43.141396,-9.854416,10.819540,-14.295117,-2.288000,-10.151420,-4.941710,-6.615477,-2.972474,...,-1.439343,0.234170,0.237684,0.759627,0.519322,0.865444,1.468920,1.691772,8,surprise
3,-553.611633,41.233486,-9.912664,8.587062,-5.737630,-5.831647,-17.839375,-11.084945,-11.369494,3.470583,...,-0.685990,0.215328,-0.507966,0.684138,0.205525,0.921709,0.126503,0.312205,6,fearful
4,-576.153442,49.947994,-13.857949,8.248825,-12.159565,0.193613,-8.719680,-7.731156,-8.692744,0.833810,...,0.724145,1.723078,1.540662,1.585126,0.513308,0.538170,-0.266717,-1.757481,5,angry


In [3]:
df.drop(inplace=True, columns=["emotions"])
df.head()

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,label
0,-573.379150,34.064041,-33.337578,3.489916,-13.482152,-6.327230,-17.734171,-15.280939,-8.379515,-0.323866,...,-0.138160,-1.091130,-2.829661,-1.520689,-2.216941,-3.051777,1.104535,0.969608,2.822009,6
1,-545.244568,38.073475,-8.023163,7.236882,-13.688817,-2.157916,-12.315373,-7.948586,-14.858138,1.719064,...,3.469327,-0.281293,0.684120,0.073632,2.641415,0.838373,1.800676,-0.601248,1.839665,5
2,-690.535461,43.141396,-9.854416,10.819540,-14.295117,-2.288000,-10.151420,-4.941710,-6.615477,-2.972474,...,0.658183,-1.439343,0.234170,0.237684,0.759627,0.519322,0.865444,1.468920,1.691772,8
3,-553.611633,41.233486,-9.912664,8.587062,-5.737630,-5.831647,-17.839375,-11.084945,-11.369494,3.470583,...,0.015738,-0.685990,0.215328,-0.507966,0.684138,0.205525,0.921709,0.126503,0.312205,6
4,-576.153442,49.947994,-13.857949,8.248825,-12.159565,0.193613,-8.719680,-7.731156,-8.692744,0.833810,...,2.204446,0.724145,1.723078,1.540662,1.585126,0.513308,0.538170,-0.266717,-1.757481,5


In [4]:
target = df["label"]

In [5]:
data = df.drop("label", axis=1)
feature_names = data.columns
data.head()

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,-573.379150,34.064041,-33.337578,3.489916,-13.482152,-6.327230,-17.734171,-15.280939,-8.379515,-0.323866,...,-4.505692,-0.138160,-1.091130,-2.829661,-1.520689,-2.216941,-3.051777,1.104535,0.969608,2.822009
1,-545.244568,38.073475,-8.023163,7.236882,-13.688817,-2.157916,-12.315373,-7.948586,-14.858138,1.719064,...,3.918904,3.469327,-0.281293,0.684120,0.073632,2.641415,0.838373,1.800676,-0.601248,1.839665
2,-690.535461,43.141396,-9.854416,10.819540,-14.295117,-2.288000,-10.151420,-4.941710,-6.615477,-2.972474,...,-1.102092,0.658183,-1.439343,0.234170,0.237684,0.759627,0.519322,0.865444,1.468920,1.691772
3,-553.611633,41.233486,-9.912664,8.587062,-5.737630,-5.831647,-17.839375,-11.084945,-11.369494,3.470583,...,0.062081,0.015738,-0.685990,0.215328,-0.507966,0.684138,0.205525,0.921709,0.126503,0.312205
4,-576.153442,49.947994,-13.857949,8.248825,-12.159565,0.193613,-8.719680,-7.731156,-8.692744,0.833810,...,0.995639,2.204446,0.724145,1.723078,1.540662,1.585126,0.513308,0.538170,-0.266717,-1.757481


In [6]:
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [7]:
y_train

1413    7
1556    7
2639    2
1189    8
2299    3
       ..
3444    6
466     6
3092    5
3772    2
860     4
Name: label, Length: 3180, dtype: int64

In [8]:
from joblib import load

X_scaler = load('scaler.sav')

In [9]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
# One-hot encoding
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()

model.add(Dense(units=256, activation='relu', input_dim=40))
model.add(Dense(units=256, activation='relu'))
model.add(Dense(units=y_train_categorical.shape[1], activation='softmax'))

In [23]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 256)               10496     
_________________________________________________________________
dense_11 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_12 (Dense)             (None, 9)                 2313      
Total params: 78,601
Trainable params: 78,601
Non-trainable params: 0
_________________________________________________________________


In [24]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [25]:
# Fit the model to the training data
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Train on 3180 samples
Epoch 1/100
3180/3180 - 1s - loss: 0.8636 - accuracy: 0.7104
Epoch 2/100
3180/3180 - 0s - loss: 0.4922 - accuracy: 0.8236
Epoch 3/100
3180/3180 - 0s - loss: 0.4154 - accuracy: 0.8509
Epoch 4/100
3180/3180 - 0s - loss: 0.3657 - accuracy: 0.8730
Epoch 5/100
3180/3180 - 0s - loss: 0.3114 - accuracy: 0.8896
Epoch 6/100
3180/3180 - 0s - loss: 0.2756 - accuracy: 0.9050
Epoch 7/100
3180/3180 - 0s - loss: 0.2405 - accuracy: 0.9176
Epoch 8/100
3180/3180 - 0s - loss: 0.2078 - accuracy: 0.9343
Epoch 9/100
3180/3180 - 0s - loss: 0.1938 - accuracy: 0.9340
Epoch 10/100
3180/3180 - 0s - loss: 0.1653 - accuracy: 0.9478
Epoch 11/100
3180/3180 - 0s - loss: 0.1420 - accuracy: 0.9528
Epoch 12/100
3180/3180 - 0s - loss: 0.1312 - accuracy: 0.9557
Epoch 13/100
3180/3180 - 0s - loss: 0.1156 - accuracy: 0.9670
Epoch 14/100
3180/3180 - 0s - loss: 0.0878 - accuracy: 0.9770
Epoch 15/100
3180/3180 - 0s - loss: 0.0825 - accuracy: 0.9755
Epoch 16/100
3180/3180 - 0s - loss: 0.0657 - accuracy: 0.